# Arquitetura de Agente Autônomo v2: Navegação Declarativa em Grafo

Este notebook implementa a versão 2 de um agente autônomo projetado para navegar em uma narrativa de RPG. A estrutura do livro-jogo funciona como um **grafo direcionado**, onde cada página é um **nó** e as escolhas são as **arestas** que conectam os nós.

O objetivo principal é testar e validar a capacidade do agente de executar um ciclo **OODA (Observe, Orient, Decide, Act)** de forma puramente declarativa:
1.  **Observar**: Ler o estado atual (a página do jogo).
2.  **Orientar-se**: Avaliar as opções com base em seu estado interno (ficha de personagem) e pré-requisitos definidos nos dados.
3.  **Decidir**: Escolher uma ação válida (atualmente, a primeira disponível).
4.  **Agir**: Executar os efeitos da escolha (rolar dados, gastar recursos, mudar de página) conforme instruído pelos dados do jogo.

Esta versão refatorada divide cada componente lógico em células separadas para maior clareza e adiciona explicações detalhadas para cada função e método.

### 1. Template da Ficha de Personagem

A função `create_character_sheet` é a fundação do estado do agente. Ela gera um dicionário Python que serve como um template em branco para a ficha de um personagem.

**Funcionamento:**
- **Estrutura Hierárquica**: A ficha é organizada em seções como `info`, `resources`, `skills`, e `status`, facilitando o acesso e a modificação de atributos específicos.
- **Valores Padrão**: Todos os campos são inicializados com valores padrão (zeros, listas vazias ou `None`). Isso garante que o agente sempre tenha uma estrutura de dados consistente, mesmo antes da personalização.
- **Pronta para Expansão**: A estrutura inclui seções como `modifiers` e `expert` skills, que são projetadas para serem preenchidas dinamicamente durante o jogo, à medida que o personagem ganha novas habilidades ou sofre penalidades.

In [ ]:
import random

def create_character_sheet():
    """Cria um template para a ficha de personagem."""
    return {
        "info": {
            "name": "Character Name",
            "occupation": None,
            "age": 30,
            "backstory": ""
        },
        "contacts": {},
        "case_files": [],
        "magic": {"spells": [], "signare": []},
        "characteristics": {
            "STR": {"full": 0, "half": 0}, "CON": {"full": 0, "half": 0},
            "DEX": {"full": 0, "half": 0}, "INT": {"full": 0, "half": 0},
            "POW": {"full": 0, "half": 0}
        },
        "resources": {
            "luck": {"starting": 0, "current": 0},
            "magic_pts": {"starting": 0, "current": 0},
            "mov": 8
        },
        "skills": {
            "common": {
                "Athletics": {"full": 30, "half": 15}, "Drive": {"full": 30, "half": 15},
                "Navigate": {"full": 30, "half": 15}, "Observation": {"full": 30, "half": 15},
                "Read Person": {"full": 30, "half": 15}, "Research": {"full": 30, "half": 15},
                "Social": {"full": 30, "half": 15}, "Stealth": {"full": 30, "half": 15},
            },
            "combat": {
                "Fighting": {"full": 30, "half": 15}, "Firearms": {"full": 30, "half": 15}
            },
            "expert": {}
        },
        "status": {
            "damage_levels": ["Hurt", "Bloodied", "Down", "Impaired"],
            "current_damage": None,
            "damage_taken": 0,
            "modifiers": []  # e.g., {"skill": "Fighting", "type": "penalty_die", "duration": "scene"}
        },
        "inventory": {"equipment": [], "weapons": []},
        "page_history": []
    }

### 2. Configuração Inicial do Personagem

A função `setup_character` personaliza a ficha de personagem criada anteriormente. Ela recebe a ficha em branco e os dados básicos do personagem (nome, ocupação) e a preenche com os valores iniciais corretos.

**Funcionamento:**
- **Dados Básicos**: Preenche as informações de identidade do personagem.
- **Recursos Iniciais**: Calcula e define recursos iniciais, como a Sorte (`luck`), usando uma rolagem de dados para adicionar variabilidade.
- **Perícias de Ocupação**: A lógica principal está em ajustar as perícias (`skills`) com base na `occupation` escolhida. Isso simula a especialização de uma classe de personagem em um RPG, dando bônus em perícias relevantes (ex: um `Police Officer` tem `Law` e `Fighting` aprimorados).

In [ ]:
def setup_character(sheet, name, occupation, backstory):
    """Configura a ficha de personagem com base na ocupação e história."""
    sheet["info"]["name"] = name
    sheet["info"]["occupation"] = occupation
    sheet["info"]["backstory"] = backstory

    # Define a sorte inicial do personagem
    luck_roll = random.randint(1, 10) + random.randint(1, 10) + 50
    sheet["resources"]["luck"]["starting"] = luck_roll
    sheet["resources"]["luck"]["current"] = luck_roll

    # Ajusta as perícias com base na ocupação
    if occupation == "Police Officer":
        for skill in ["Law", "Social", "Athletics", "Fighting"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Law"] = {"full": 60, "half": 30}
    elif occupation == "Social Worker":
        for skill in ["Observation", "Research", "Social"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}
    elif occupation == "Nurse":
        for skill in ["Observation", "Read Person", "Social"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Medicine"] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}

    return sheet

### 3. Mecânica de Rolagem de Dados (D100)

A função `make_check` é o motor de resolução de ações do RPG. Ela simula uma rolagem de dado de 100 lados (D100) e determina o nível de sucesso de uma ação com base no valor de uma perícia.

**Funcionamento:**
- **Rolagem D100**: Simula uma rolagem de percentil combinando um dado de dezenas e um de unidades.
- **Bônus e Penalidade**: Implementa a mecânica de "vantagem" e "desvantagem". Com um dado de bônus (`bonus_dice`), o menor de dois dados de dezena é usado. Com um dado de penalidade (`penalty_die`), o maior é usado. Se ambos estiverem presentes, eles se anulam.
- **Níveis de Sucesso**: O resultado da rolagem é comparado com o valor da perícia (`target_value`) e sua metade (`half_value`) para determinar um de cinco níveis de sucesso, retornados como um valor numérico:
    - **5**: Sucesso Crítico (rolagem 1)
    - **4**: Sucesso Difícil (rolagem <= metade da perícia)
    - **3**: Sucesso Normal (rolagem <= perícia)
    - **2**: Falha (rolagem > perícia)
    - **1**: Falha Crítica (rolagem 100)
- **Saída Declarativa**: O retorno numérico (`level`) permite que o sistema encontre a consequência correta de forma declarativa, sem precisar de lógica complexa no código que chama a função.

In [ ]:
def make_check(target_value, half_value, bonus_dice=False, penalty_die=False):
    """Realiza um teste de perícia D100 e retorna o nível de sucesso numérico."""
    tens_roll_1 = random.randint(0, 9) * 10
    tens_roll_2 = random.randint(0, 9) * 10
    units_roll = random.randint(1, 10)

    # Um dado de bônus e um dado de penalidade se anulam.
    if bonus_dice and penalty_die:
        bonus_dice = False
        penalty_die = False

    if bonus_dice:
        final_tens = min(tens_roll_1, tens_roll_2)
        print("Applied bonus die.")
    elif penalty_die:
        final_tens = max(tens_roll_1, tens_roll_2)
        print("Applied penalty die.")
    else:
        final_tens = tens_roll_1

    # Calcula o resultado final da rolagem
    if final_tens == 0 and units_roll == 10:
        final_roll = 100
    elif final_tens == 0:
        final_roll = units_roll
    else:
        final_roll = final_tens + (units_roll % 10)

    # Determina o nível de sucesso
    if final_roll == 1:
        return (5, final_roll)  # Critical Success
    if final_roll == 100:
        return (1, final_roll)  # Fumble
    if final_roll <= half_value:
        return (4, final_roll)  # Hard Success
    if final_roll <= target_value:
        return (3, final_roll)  # Success
    
    return (2, final_roll)  # Failure

### 4. A Classe `Agent`: O Coração do Sistema

A classe `Agent` encapsula a lógica de decisão e ação, implementando o ciclo OODA. Ela funciona como o "cérebro" do jogador autônomo.

**Estrutura da Classe:**
- **`__init__`**: O construtor inicializa o agente. Ele cria a ficha de personagem chamando `create_character_sheet` e `setup_character`, e armazena os dados do jogo (`game_data`) e as instruções (`game_instructions`) recebidas via injeção de dependência.
- **`_llm_decide` (Decisão)**: Simula a tomada de decisão. Em uma implementação completa, aqui entraria um Modelo de Linguagem (LLM). Atualmente, a lógica verifica os pré-requisitos de cada escolha (como custo de sorte ou magia) e seleciona a primeira opção válida e disponível.
- **`perform_action` (Ação)**: Executa a escolha decidida. Esta função é um interpretador declarativo: ela lê as instruções da escolha (como `roll`, `opposed_roll`, `effects`) e chama as funções correspondentes (`make_check`, `apply_damage`, etc.) para alterar o estado do jogo e do personagem.
- **`run` (Loop Principal)**: Orquestra o ciclo OODA. A cada iteração, ele obtém a página atual (**Observação**), chama `_llm_decide` (**Orientação/Decisão**), e usa `perform_action` para avançar a história (**Ação**).
- **Métodos Auxiliares**: Funções como `apply_damage`, `spend_luck`, e `apply_penalty` gerenciam o estado interno do agente, modificando a ficha de personagem de forma controlada.

In [ ]:
class Agent:
    """
    Implementa um agente autônomo que navega por um livro-jogo
    seguindo o ciclo OODA (Observe, Orient, Decide, Act).
    """
    
    def __init__(self, name, occupation, game_instructions, game_data):
        """
        Inicializa o agente, criando sua ficha de personagem e configurando o estado inicial.
        """
        base_sheet = create_character_sheet()
        self.sheet = setup_character(base_sheet, name, occupation, game_instructions.get_backstory())
        self.game_data = game_data
        self.current_page = 1
        self.combat_status = {}

    def __repr__(self):
        """Retorna uma representação textual do agente."""
        return f"Agent(Name: {self.sheet['info']['name']}, Occupation: {self.sheet['info']['occupation']})"

    def _llm_decide(self, page_text, choices):
        """
        Fase de Decisão (OODA): Avalia as escolhas disponíveis e seleciona uma.
        Atualmente, usa lógica simples, mas é o ponto de integração para um LLM.
        """
        print("--- AGENT DECISION ---")
        print(f"Based on the situation: '{page_text[:100]}...'")
        
        valid_choices = []
        for choice in choices:
            new_choice = choice.copy()
            unavailable_reason = None

            # 1. Orientação: Verificar pré-requisitos de recursos (custos)
            if "effects" in new_choice:
                for effect in new_choice["effects"]:
                    if effect["action"] == "spend_luck":
                        if self.sheet["resources"]["luck"]["current"] < effect["amount"]:
                            unavailable_reason = f"Custo de Sorte ({effect['amount']}) é maior que o atual ({self.sheet['resources']['luck']['current']})"
                            break
                    elif effect["action"] == "spend_magic":
                        if self.sheet["resources"]["magic_pts"]["current"] < effect["amount"]:
                            unavailable_reason = f"Custo de Magia ({effect['amount']}) é maior que o atual ({self.sheet['resources']['magic_pts']['current']})"
                            break
            
            # 2. Orientação: Verificar pré-requisitos de estado (condições)
            if not unavailable_reason and "requires" in new_choice:
                requirements = new_choice["requires"]
                
                # Checar ocupação
                if "occupation" in requirements:
                    if self.sheet["info"]["occupation"] != requirements["occupation"]:
                        unavailable_reason = f"Requer ocupação: {requirements['occupation']}"

                # Checar dano
                if not unavailable_reason and "damage_taken" in requirements:
                    damage_req = requirements["damage_taken"]
                    damage_taken = self.sheet["status"]["damage_taken"]
                    min_damage = damage_req.get("min", 0)
                    max_damage = damage_req.get("max", float('inf'))
                    if not (min_damage <= damage_taken <= max_damage):
                        unavailable_reason = f"Requer dano entre {min_damage} e {max_damage}"

            if unavailable_reason:
                new_choice["unavailable"] = True
                new_choice["unavailable_reason"] = unavailable_reason

            valid_choices.append(new_choice)

        if not valid_choices:
            print("No valid choices found for the agent.")
            return None

        print("Available choices:")
        for choice in valid_choices:
            log_message = choice.get('text', f"Declarative action: {list(choice.keys())}")
            if choice.get("unavailable"):
                print(f"- {log_message} (Indisponível: {choice['unavailable_reason']})")
            else:
                print(f"- {log_message}")

        # 3. Decisão: Escolher a primeira opção disponível
        available_choices = [c for c in valid_choices if not c.get("unavailable")]
        
        if not available_choices:
            print("No available choices for the agent to take.")
            return None

        chosen = available_choices[0].copy()
        log_message = chosen.get('text', f"Declarative action: {list(chosen.keys())}")
        print(f"Agent chose: '{log_message}'")
        return chosen

    def perform_action(self, choice):
        """
        Fase de Ação (OODA): Processa a escolha e retorna o ID da próxima página.
        Funciona como um interpretador para a estrutura de dados da escolha.
        """
        if not choice:
            return None

        # 1. Efeitos de Custo: Processar gastos de recursos ANTES da ação.
        if "effects" in choice:
            for effect in choice["effects"]:
                if effect["action"] == "spend_luck": self.spend_luck(effect["amount"])
                elif effect["action"] == "spend_magic": self.spend_magic(effect["amount"])

        # 2. Ação Principal: Determinar o tipo de ação e executá-la.
        if "roll" in choice:
            skill_to_roll = choice["roll"].split(" ")[0]
            has_penalty = any(mod["skill"] == skill_to_roll and mod["type"] == "penalty_die" for mod in self.sheet["status"]["modifiers"])
            skill_values = self.sheet["skills"]["common"].get(skill_to_roll) or \
                           self.sheet["skills"]["combat"].get(skill_to_roll) or \
                           self.sheet["skills"]["expert"].get(skill_to_roll) or \
                           self.sheet["characteristics"].get(skill_to_roll, {"full": 30, "half": 15})

            level, roll_value = make_check(skill_values["full"], skill_values["half"], bonus_dice=choice.get("bonus_dice", False), penalty_die=has_penalty)
            print(f"Agent rolled for {skill_to_roll}: {roll_value} -> Level {level}")

            # Busca hierárquica pelo resultado correspondente ao nível de sucesso.
            result_outcome = choice["results"].get(str(level)) or choice["results"].get("2") # Default para falha
            
            if "effects" in result_outcome:
                for effect in result_outcome["effects"]:
                    if effect["action"] == "take_damage": self.apply_damage(effect["amount"])
                    elif effect["action"] == "apply_penalty": self.apply_penalty(effect["skill"], effect["duration"])
            
            return result_outcome.get("goto")

        if "opposed_roll" in choice:
            # Lógica para testes resistidos entre o agente e um oponente.
            agent_skill_name = choice["opposed_roll"]
            agent_has_penalty = any(mod["skill"] == agent_skill_name and mod["type"] == "penalty_die" for mod in self.sheet["status"]["modifiers"])
            agent_skill_values = self.sheet["skills"]["combat"].get(agent_skill_name, {"full": 30, "half": 15})
            agent_level, agent_roll = make_check(agent_skill_values["full"], agent_skill_values["half"], penalty_die=agent_has_penalty)
            print(f"Agent's Roll ({agent_skill_name}): {agent_roll} -> Level {agent_level}")

            opponent_skill_values = choice["opponent_skill"]
            opponent_has_penalty = choice.get("opponent_penalty", False)
            opponent_level, opponent_roll = make_check(opponent_skill_values["full"], opponent_skill_values["half"], penalty_die=opponent_has_penalty)
            print(f"Opponent's Roll: {opponent_roll} -> Level {opponent_level}")

            if agent_level > opponent_level: outcome_key = "win"
            elif agent_level < opponent_level: outcome_key = "lose"
            else: outcome_key = "draw"
            
            result_outcome = choice["outcomes"][outcome_key]
            
            if "effects" in result_outcome:
                for effect in result_outcome["effects"]:
                    if effect["action"] == "take_damage": self.apply_damage(effect["amount"])
            
            return result_outcome.get("goto")

        if "luck_roll" in choice:
            # Lógica para testes de sorte.
            roll = random.randint(1, 100)
            result_outcome = choice["results"]["3"] if roll <= self.sheet["resources"]["luck"]["current"] else choice["results"]["2"]
            
            if "effects" in result_outcome:
                for effect in result_outcome["effects"]:
                    if effect["action"] == "take_damage": self.apply_damage(effect["amount"])

            return result_outcome.get("goto")

        # Ação padrão: simplesmente ir para a próxima página.
        return choice.get("goto")

    def run(self):
        """
        Motor do jogo: executa o loop principal do agente.
        """
        while self.current_page is not None:
            # Condição de parada: personagem incapacitado.
            if self.sheet["status"]["current_damage"] == "Impaired":
                print("\n--- CHARACTER IS IMPAIRED ---")
                print("The character has taken 4 or more damage and can no longer continue.")
                break

            self.sheet["page_history"].append(self.current_page)
            
            # 1. Observação: Obter dados da página atual.
            page = self.game_data.get_page(self.current_page)
            if not page:
                print(f"Page {self.current_page} not found. Ending game.")
                break

            print(f"\n--- PAGE {self.current_page} ---")
            print(page["text"])

            if not page["choices"]:
                print("\n--- END OF STORY ---")
                break

            # Limpar modificadores de estado temporários (duração "scene").
            self.sheet["status"]["modifiers"] = [
                mod for mod in self.sheet["status"]["modifiers"]
                if mod.get("duration") != "scene"
            ]

            # 2. Decisão: Chamar o método de decisão.
            choice = self._llm_decide(page["text"], page["choices"])
            
            if choice is None:
                print("Agent cannot make a move. Staying on the current page.")
                continue # Permanece na página atual se nenhuma ação for possível.

            # 3. Ação: Executar a escolha e obter a próxima página.
            self.current_page = self.perform_action(choice)
            
            if self.current_page is None:
                print("Agent reached a dead end.")

    # --- Métodos Auxiliares de Gerenciamento de Estado ---

    def spend_luck(self, amount):
        """Gasta pontos de sorte."""
        if self.sheet["resources"]["luck"]["current"] >= amount:
            self.sheet["resources"]["luck"]["current"] -= amount
            print(f"Spent {amount} luck. Current luck: {self.sheet['resources']['luck']['current']}")
            return True
        return False

    def spend_magic(self, cost):
        """Gasta pontos de magia."""
        if self.sheet["resources"]["magic_pts"]["current"] >= cost:
            self.sheet["resources"]["magic_pts"]["current"] -= cost
            print(f"Spent {cost} magic points. Current magic points: {self.sheet['resources']['magic_pts']['current']}")
            return True
        return False

    def apply_damage(self, amount):
        """Aplica dano e atualiza o status do personagem."""
        self.sheet["status"]["damage_taken"] += amount
        total_damage = self.sheet["status"]["damage_taken"]
        
        new_status = None
        if total_damage >= 4: new_status = "Impaired"
        elif total_damage == 3: new_status = "Down"
        elif total_damage == 2: new_status = "Bloodied"
        elif total_damage == 1: new_status = "Hurt"
        
        if new_status and self.sheet["status"]["current_damage"] != new_status:
            self.sheet["status"]["current_damage"] = new_status
            print(f"Character took {amount} damage. Total damage: {total_damage}. Status is now {new_status}.")
        else:
            print(f"Character took {amount} damage. Total damage: {total_damage}.")

    def apply_penalty(self, skill, duration):
        """Aplica uma penalidade a uma perícia."""
        modifier = {"skill": skill, "type": "penalty_die", "duration": duration}
        self.sheet["status"]["modifiers"].append(modifier)
        print(f"Applied penalty to {skill} for {duration}.")

    def heal_status(self, status_to_heal):
        """Cura dano ou remove um status de penalidade."""
        if status_to_heal == "Hurt" and self.sheet["status"]["damage_taken"] > 0:
            self.sheet["status"]["damage_taken"] -= 1
            print("Healed 1 point of damage.")
            # Atualiza o status de dano com base no novo total.
            total_damage = self.sheet["status"]["damage_taken"]
            new_status = None
            if total_damage >= 4: new_status = "Impaired"
            elif total_damage == 3: new_status = "Down"
            elif total_damage == 2: new_status = "Bloodied"
            elif total_damage == 1: new_status = "Hurt"
            self.sheet["status"]["current_damage"] = new_status


### 5. Ponto de Entrada: Executando o Jogo

Este bloco de código é o ponto de entrada (`main`) do script. É aqui que todos os componentes são montados para iniciar a simulação.

**Funcionamento:**
1.  **Instanciação dos Dados**: Os objetos `GameData` (que carrega `pages.py`) e `GameInstructions` (com o backstory) são criados.
2.  **Criação do Agente**: Um `Agent` é instanciado com um nome, uma ocupação e as dependências de dados. A variável `player_occupation` pode ser alterada para testar como o agente se comporta com diferentes conjuntos de perícias iniciais.
3.  **Execução**: O método `agent.run()` é chamado, dando início ao loop principal do jogo e permitindo que o agente comece sua jornada pela narrativa.

In [ ]:
# Este bloco só é executado quando o script é rodado diretamente.
if __name__ == '__main__':
    # Dependências externas (assumindo que estes arquivos existem)
    # from pages import GameData
    # from instructions import GameInstructions

    # Mock objects para permitir a execução sem os arquivos externos
    class MockGameData:
        def get_page(self, page_id):
            # Adicione páginas de teste aqui se necessário
            return None 
    class MockGameInstructions:
        def get_backstory(self):
            return "Um agente em uma missão de teste."

    # 1. Instanciar os dados e as instruções do jogo
    # game_data = GameData()
    # game_instructions = GameInstructions()
    game_data = MockGameData()
    game_instructions = MockGameInstructions()

    # 2. Criar o agente, injetando as dependências
    player_name = "Alex"
    player_occupation = "Police Officer" # Mude para "Social Worker" ou "Nurse"
    agent = Agent(player_name, player_occupation, game_instructions, game_data)

    # 3. Rodar o loop principal do jogo
    print(f"Starting agent run for {agent}")
    # agent.run() # Comentado para evitar loop infinito sem dados de página
    print("Agent run finished (mocked).")